In [1]:
import pandas as pd
fp = "../../data/incident_event_log_02.csv"
df = pd.read_csv(fp)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_2987/1966600636.py:3: DtypeWarning: Columns (19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


In [2]:
df_size = pd.DataFrame.from_records([{"rows": df.shape[0], "cols": df.shape[1]}])

In [3]:
col_df = pd.DataFrame({"columns": df.columns.tolist()})

In [4]:
df_nv = df.isnull().any().to_frame().reset_index()
df_nv.columns = ["column", "null_values"]
df_nv.to_json()

'{"column":{"0":"number","1":"incident_state","2":"active","3":"reassignment_count","4":"reopen_count","5":"sys_mod_count","6":"made_sla","7":"caller_id","8":"opened_by","9":"opened_at","10":"sys_created_by","11":"sys_created_at","12":"sys_updated_by","13":"sys_updated_at","14":"contact_type","15":"location","16":"category","17":"subcategory","18":"u_symptom","19":"cmdb_ci","20":"impact","21":"urgency","22":"priority","23":"assignment_group","24":"assigned_to","25":"knowledge","26":"u_priority_confirmation","27":"notify","28":"problem_id","29":"rfc","30":"vendor","31":"caused_by","32":"closed_code","33":"resolved_by","34":"resolved_at","35":"closed_at"},"null_values":{"0":false,"1":false,"2":false,"3":false,"4":false,"5":false,"6":false,"7":true,"8":true,"9":false,"10":true,"11":true,"12":false,"13":false,"14":false,"15":true,"16":true,"17":true,"18":true,"19":true,"20":false,"21":false,"22":false,"23":true,"24":true,"25":false,"26":false,"27":false,"28":true,"29":true,"30":true,"31":t

In [5]:
df_nv

,column,null_values
0,number,False
1,incident_state,False
2,active,False
3,reassignment_count,False
4,reopen_count,False
5,sys_mod_count,False
6,made_sla,False
7,caller_id,True
8,opened_by,True
9,opened_at,False


In [6]:
df_raw_types = df.dtypes.to_frame().reset_index()
df_raw_types.columns = ["column", "dtype"]
df_raw_types["dtype"] = df_raw_types["dtype"].astype(str)
df_raw_summary = df_raw_types.merge(df_nv, on="column")
df_raw_summary

,column,dtype,null_values
0,number,object,False
1,incident_state,object,False
2,active,bool,False
3,reassignment_count,int64,False
4,reopen_count,int64,False
5,sys_mod_count,int64,False
6,made_sla,bool,False
7,caller_id,object,True
8,opened_by,object,True
9,opened_at,object,False


In [7]:
cols = df.columns.tolist()
int_cols = [c for c in cols if "count" in c]
date_time_cols = [c for c in cols if "_at" in c]


In [8]:
raw_type_info = df.dtypes.to_dict()
mod_type_info = raw_type_info.copy()

In [9]:
for k,v in mod_type_info.items():
    if k in int_cols:
        mod_type_info[k] = "int64"
    if k in date_time_cols:
        mod_type_info[k] = "datetime64[ns]"
    

In [10]:
from datetime import datetime
nan_date_str = "1900-01-01 00:00:00"
nan_date_time = datetime.strptime(nan_date_str, "%Y-%m-%d %H:%M:%S")

In [11]:
missing_dates = df["sys_created_at"].isnull()

In [12]:
for dt_col in date_time_cols:
    df[dt_col] = df[dt_col].fillna(nan_date_time)
    df[dt_col] = pd.to_datetime(df[dt_col], dayfirst=True)

In [13]:
category_cols = [ c for c in cols if (c not in int_cols and c not in date_time_cols)]

In [14]:
category_cols

['number',
 'incident_state',
 'active',
 'made_sla',
 'caller_id',
 'opened_by',
 'sys_created_by',
 'sys_updated_by',
 'contact_type',
 'location',
 'category',
 'subcategory',
 'u_symptom',
 'cmdb_ci',
 'impact',
 'urgency',
 'priority',
 'assignment_group',
 'assigned_to',
 'knowledge',
 'u_priority_confirmation',
 'notify',
 'problem_id',
 'rfc',
 'vendor',
 'caused_by',
 'closed_code',
 'resolved_by']

In [15]:
nan_strings = "UNKNOWN"
for c in category_cols:
    df[c] = df[c].fillna("UNKNOWN")

In [16]:
df_nv_fix_check = df.isnull().any().to_frame().reset_index()
df_nv_fix_check.columns = ["columns", "null_values"]

In [17]:
df_nv_fix_check

,columns,null_values
0,number,False
1,incident_state,False
2,active,False
3,reassignment_count,False
4,reopen_count,False
5,sys_mod_count,False
6,made_sla,False
7,caller_id,False
8,opened_by,False
9,opened_at,False


In [18]:
mod_type_info

{'number': dtype('O'),
 'incident_state': dtype('O'),
 'active': dtype('bool'),
 'reassignment_count': 'int64',
 'reopen_count': 'int64',
 'sys_mod_count': 'int64',
 'made_sla': dtype('bool'),
 'caller_id': dtype('O'),
 'opened_by': dtype('O'),
 'opened_at': 'datetime64[ns]',
 'sys_created_by': dtype('O'),
 'sys_created_at': 'datetime64[ns]',
 'sys_updated_by': dtype('O'),
 'sys_updated_at': 'datetime64[ns]',
 'contact_type': dtype('O'),
 'location': dtype('O'),
 'category': dtype('O'),
 'subcategory': dtype('O'),
 'u_symptom': dtype('O'),
 'cmdb_ci': dtype('O'),
 'impact': dtype('O'),
 'urgency': dtype('O'),
 'priority': dtype('O'),
 'assignment_group': dtype('O'),
 'assigned_to': dtype('O'),
 'knowledge': dtype('bool'),
 'u_priority_confirmation': dtype('bool'),
 'notify': dtype('O'),
 'problem_id': dtype('O'),
 'rfc': dtype('O'),
 'vendor': dtype('O'),
 'caused_by': dtype('O'),
 'closed_code': dtype('O'),
 'resolved_by': dtype('O'),
 'resolved_at': 'datetime64[ns]',
 'closed_at': 'd

In [19]:
df = df.astype(mod_type_info)

In [20]:
df.dtypes

number                             object
incident_state                     object
active                               bool
reassignment_count                  int64
reopen_count                        int64
sys_mod_count                       int64
made_sla                             bool
caller_id                          object
opened_by                          object
opened_at                  datetime64[ns]
sys_created_by                     object
sys_created_at             datetime64[ns]
sys_updated_by                     object
sys_updated_at             datetime64[ns]
contact_type                       object
location                           object
category                           object
subcategory                        object
u_symptom                          object
cmdb_ci                            object
impact                             object
urgency                            object
priority                           object
assignment_group                  

In [21]:
import tables

In [22]:
root_table_desc = df.dtypes.to_dict()

In [23]:
type(root_table_desc)

dict

In [24]:
proc_summ = [{"Step": 1 , "Description": "Profile the types in the raw dataset"},
             {"Step": 2, "Description": "Profile the missing values in the raw dataset"},
{"Step": 3, "Description": "Fix the missing values in the raw dataset based on data type"},
{"Step": 4, "Description": "Assign the correct data types to each column"},
{"Step": 5, "Description": "Prepare processing summary"},
{"Step": 6, "Description": "Write HDF file" }]

In [25]:
df_ps = pd.DataFrame.from_dict(proc_summ)

In [26]:
fpm = "../../data/meta_data/ITSM_dataset.h5"
store = pd.HDFStore(fpm)
store['raw_dataset'] = df
store['raw_dataset/raw_table_issues'] = df_raw_summary
store['raw_dataset/raw_table_processing_summary'] = df_raw_summary
store.close()